# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** Our CNN-RNN architecture is similar to that proposed in the NIC (Neural Image Caption) [paper](https://arxiv.org/pdf/1411.4555.pdf), which combines deep convolutional nets (resnet50) for image classification with recurrent networks for sequence modeling (LSTM) to create a single joint network that generates descriptions of images. The hyperparameters are given based on my experience as well as referring to [the_paper](http://cs231n.stanford.edu/reports/2017/pdfs/709.pdf). hidden_size was set as 512, since it needs a large vector to capture the essence of the image info; but later it was set as 256, since the LSTM has two layers.  


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I left the transform as it is provided in the project because the transform randomly crop, horizontally flp, and normalize the input image, which contain the common preprocessing procedures. 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** The trainable parameters consist of the last layer of the encoder and all the layers of the decoder. Since resnet50 is well pretrained, only the last linear layer needs to be trained.  

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** Adam is chosen since it is generally a good optimizer. 

In [7]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256 #128           # dimensionality of image and word embeddings
hidden_size = 256 #512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.0001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...





 18%|█▊        | 75063/414113 [00:26<01:59, 2848.78it/s]


  0%|          | 0/414113 [00:00<?, ?it/s]


  0%|          | 417/414113 [00:00<01:39, 4168.90it/s]


  0%|          | 897/414113 [00:00<01:35, 4339.43it/s]

Done (t=0.87s)
creating index...
index created!
Obtaining caption lengths...





  0%|          | 1378/414113 [00:00<01:32, 4468.55it/s]


  0%|          | 1847/414113 [00:00<01:30, 4531.56it/s]


  1%|          | 2311/414113 [00:00<01:30, 4562.44it/s]


  1%|          | 2783/414113 [00:00<01:29, 4607.34it/s]


  1%|          | 3196/414113 [00:00<01:32, 4429.20it/s]


  1%|          | 3675/414113 [00:00<01:30, 4530.07it/s]


  1%|          | 4147/414113 [00:00<01:29, 4582.36it/s]


  1%|          | 4628/414113 [00:01<01:28, 4645.70it/s]


  1%|          | 5119/414113 [00:01<01:26, 4720.08it/s]


  1%|▏         | 5601/414113 [00:01<01:26, 4749.22it/s]


  1%|▏         | 6072/414113 [00:01<01:26, 4733.61it/s]


  2%|▏         | 6542/414113 [00:01<01:26, 4721.02it/s]


  2%|▏         | 7014/414113 [00:01<01:26, 4718.63it/s]


  2%|▏         | 7491/414113 [00:01<01:25, 4731.63it/s]


  2%|▏         | 7981/414113 [00:01<01:24, 4778.07it/s]


  2%|▏         | 8475/414113 [00:01<01:24, 4824.54it/s]


  2%|▏         | 8959/414113 [00:01<01:23, 4827.73it/s]


  2%|▏     

 14%|█▍        | 58911/414113 [00:12<01:14, 4779.96it/s]


 14%|█▍        | 59396/414113 [00:12<01:13, 4798.08it/s]


 14%|█▍        | 59876/414113 [00:12<01:13, 4798.25it/s]


 15%|█▍        | 60366/414113 [00:13<01:13, 4825.78it/s]


 15%|█▍        | 60849/414113 [00:13<01:13, 4796.04it/s]


 15%|█▍        | 61349/414113 [00:13<01:12, 4854.59it/s]


 15%|█▍        | 61835/414113 [00:13<01:13, 4817.48it/s]


 15%|█▌        | 62317/414113 [00:13<01:13, 4773.01it/s]


 15%|█▌        | 62807/414113 [00:13<01:13, 4808.13it/s]


 15%|█▌        | 63289/414113 [00:13<01:13, 4774.03it/s]


 15%|█▌        | 63775/414113 [00:13<01:13, 4797.76it/s]


 16%|█▌        | 64255/414113 [00:13<01:13, 4784.58it/s]


 16%|█▌        | 64734/414113 [00:13<01:13, 4770.80it/s]


 16%|█▌        | 65212/414113 [00:14<01:13, 4761.73it/s]


 16%|█▌        | 65689/414113 [00:14<01:13, 4758.39it/s]


 16%|█▌        | 66174/414113 [00:14<01:12, 4783.90it/s]


 16%|█▌        | 66653/414113 [00:14<01:12, 4775.89it/s]

...

 42%|████▏     | 173265/414113 [00:36<00:52, 4622.32it/s]


 42%|████▏     | 173736/414113 [00:36<00:51, 4645.94it/s]


 42%|████▏     | 174201/414113 [00:36<00:51, 4639.06it/s]


 42%|████▏     | 174670/414113 [00:37<00:51, 4653.73it/s]


 42%|████▏     | 175136/414113 [00:37<00:51, 4644.26it/s]


 42%|████▏     | 175605/414113 [00:37<00:51, 4655.08it/s]


 43%|████▎     | 176085/414113 [00:37<00:50, 4695.64it/s]


 43%|████▎     | 176555/414113 [00:37<00:50, 4677.20it/s]


 43%|████▎     | 177023/414113 [00:37<00:52, 4480.70it/s]


 43%|████▎     | 177508/414113 [00:37<00:51, 4583.22it/s]


 43%|████▎     | 177973/414113 [00:37<00:51, 4600.82it/s]


 43%|████▎     | 178442/414113 [00:37<00:50, 4625.07it/s]


 43%|████▎     | 178918/414113 [00:38<00:50, 4661.48it/s]


 43%|████▎     | 179385/414113 [00:38<01:42, 2289.68it/s]


 43%|████▎     | 179840/414113 [00:38<01:27, 2690.54it/s]


 44%|████▎     | 180321/414113 [00:38<01:15, 3099.96it/s]


 44%|████▎     | 180794/414113 [00:38<01

 56%|█████▌    | 229989/414113 [00:49<00:38, 4807.18it/s]


 56%|█████▌    | 230470/414113 [00:49<00:38, 4769.69it/s]


 56%|█████▌    | 230955/414113 [00:49<00:38, 4791.19it/s]


 56%|█████▌    | 231435/414113 [00:49<00:38, 4731.50it/s]


 56%|█████▌    | 231909/414113 [00:49<00:39, 4627.76it/s]


 56%|█████▌    | 232378/414113 [00:49<00:39, 4645.66it/s]


 56%|█████▌    | 232844/414113 [00:49<00:39, 4631.19it/s]


 56%|█████▋    | 233308/414113 [00:49<00:40, 4483.68it/s]


 56%|█████▋    | 233790/414113 [00:49<00:39, 4577.65it/s]


 57%|█████▋    | 234269/414113 [00:49<00:38, 4638.50it/s]


 57%|█████▋    | 234742/414113 [00:50<00:38, 4662.44it/s]


 57%|█████▋    | 235219/414113 [00:50<00:38, 4692.96it/s]


 57%|█████▋    | 235702/414113 [00:50<00:37, 4732.16it/s]


 57%|█████▋    | 236194/414113 [00:50<00:37, 4786.47it/s]


 57%|█████▋    | 236674/414113 [00:50<00:37, 4764.72it/s]


 57%|█████▋    | 237152/414113 [00:50<00:37, 4766.51it/s]


 57%|█████▋    | 237632/414113 [00:50<00

 69%|██████▉   | 286630/414113 [01:00<00:26, 4735.16it/s]


 69%|██████▉   | 287116/414113 [01:01<00:26, 4769.74it/s]


 69%|██████▉   | 287594/414113 [01:01<00:26, 4754.91it/s]


 70%|██████▉   | 288079/414113 [01:01<00:26, 4782.73it/s]


 70%|██████▉   | 288561/414113 [01:01<00:26, 4792.16it/s]


 70%|██████▉   | 289046/414113 [01:01<00:26, 4808.88it/s]


 70%|██████▉   | 289532/414113 [01:01<00:25, 4823.07it/s]


 70%|███████   | 290015/414113 [01:01<00:25, 4783.84it/s]


 70%|███████   | 290500/414113 [01:01<00:25, 4802.70it/s]


 70%|███████   | 290981/414113 [01:01<00:25, 4785.38it/s]


 70%|███████   | 291460/414113 [01:01<00:25, 4780.20it/s]


 70%|███████   | 291948/414113 [01:02<00:25, 4807.25it/s]


 71%|███████   | 292433/414113 [01:02<00:25, 4819.47it/s]


 71%|███████   | 292916/414113 [01:02<00:25, 4813.36it/s]


 71%|███████   | 293409/414113 [01:02<00:24, 4846.52it/s]


 71%|███████   | 293894/414113 [01:02<00:24, 4815.25it/s]


 71%|███████   | 294376/414113 [01:02<00

 83%|████████▎ | 343284/414113 [01:12<00:14, 4737.70it/s]


 83%|████████▎ | 343762/414113 [01:12<00:14, 4747.86it/s]


 83%|████████▎ | 344246/414113 [01:13<00:14, 4774.47it/s]


 83%|████████▎ | 344727/414113 [01:13<00:14, 4784.71it/s]


 83%|████████▎ | 345206/414113 [01:13<00:14, 4734.35it/s]


 83%|████████▎ | 345682/414113 [01:13<00:14, 4740.76it/s]


 84%|████████▎ | 346157/414113 [01:13<00:14, 4739.01it/s]


 84%|████████▎ | 346632/414113 [01:13<00:31, 2172.46it/s]


 84%|████████▍ | 347099/414113 [01:14<00:25, 2587.15it/s]


 84%|████████▍ | 347580/414113 [01:14<00:22, 3003.00it/s]


 84%|████████▍ | 348053/414113 [01:14<00:19, 3371.41it/s]


 84%|████████▍ | 348517/414113 [01:14<00:17, 3670.83it/s]


 84%|████████▍ | 348986/414113 [01:14<00:16, 3925.75it/s]


 84%|████████▍ | 349465/414113 [01:14<00:15, 4149.11it/s]


 85%|████████▍ | 349944/414113 [01:14<00:14, 4322.50it/s]


 85%|████████▍ | 350414/414113 [01:14<00:14, 4428.20it/s]


 85%|████████▍ | 350906/414113 [01:14<00

 97%|█████████▋| 399693/414113 [01:25<00:03, 4749.75it/s]


 97%|█████████▋| 400169/414113 [01:25<00:03, 4610.89it/s]


 97%|█████████▋| 400648/414113 [01:25<00:02, 4663.11it/s]


 97%|█████████▋| 401123/414113 [01:25<00:02, 4688.29it/s]


 97%|█████████▋| 401607/414113 [01:25<00:02, 4731.68it/s]


 97%|█████████▋| 402082/414113 [01:25<00:02, 4735.00it/s]


 97%|█████████▋| 402559/414113 [01:25<00:02, 4744.52it/s]


 97%|█████████▋| 403034/414113 [01:25<00:02, 4738.46it/s]


 97%|█████████▋| 403509/414113 [01:25<00:02, 4716.88it/s]


 98%|█████████▊| 403981/414113 [01:26<00:02, 4712.93it/s]


 98%|█████████▊| 404458/414113 [01:26<00:02, 4728.40it/s]


 98%|█████████▊| 404931/414113 [01:26<00:01, 4704.44it/s]


 98%|█████████▊| 405402/414113 [01:26<00:01, 4704.64it/s]


 98%|█████████▊| 405873/414113 [01:26<00:01, 4702.35it/s]


 98%|█████████▊| 406349/414113 [01:26<00:01, 4717.61it/s]


 98%|█████████▊| 406821/414113 [01:26<00:01, 4712.35it/s]


 98%|█████████▊| 407293/414113 [01:26<00

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [8]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/3236], Loss: 5.4411, Perplexity: 230.7067
Epoch [1/3], Step [200/3236], Loss: 4.8672, Perplexity: 129.9574
Epoch [1/3], Step [300/3236], Loss: 4.7660, Perplexity: 117.4437
Epoch [1/3], Step [400/3236], Loss: 4.5879, Perplexity: 98.28641
Epoch [1/3], Step [500/3236], Loss: 4.5661, Perplexity: 96.17166
Epoch [1/3], Step [600/3236], Loss: 4.5517, Perplexity: 94.79096
Epoch [1/3], Step [700/3236], Loss: 4.4708, Perplexity: 87.42552
Epoch [1/3], Step [800/3236], Loss: 4.2492, Perplexity: 70.05054
Epoch [1/3], Step [900/3236], Loss: 4.1966, Perplexity: 66.45809
Epoch [1/3], Step [1000/3236], Loss: 4.4880, Perplexity: 88.9460
Epoch [1/3], Step [1100/3236], Loss: 4.0592, Perplexity: 57.92587
Epoch [1/3], Step [1200/3236], Loss: 4.0318, Perplexity: 56.36023
Epoch [1/3], Step [1300/3236], Loss: 4.0299, Perplexity: 56.25806
Epoch [1/3], Step [1400/3236], Loss: 3.8715, Perplexity: 48.01348
Epoch [1/3], Step [1500/3236], Loss: 4.1100, Perplexity: 60.94942
Epoch [1/3], Step [1

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.